In [31]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
import scipy
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
%matplotlib inline
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 22})

# Background 

This is a credit card fraud dataset. The data has already been transformed into Principal Components. The datasets contains transactions made by credit cards in September 2013 by European cardholders. This dataset presents transactions that occurred in two days, where there are 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

The goal of this assignment is to design an algorithm that can correctly identify the fraudulent cases. 
The dataset can be found here: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [2]:
fraud_data = pd.read_csv('creditcard.csv')

In [3]:
# Look at what the shape of the data looks like 
fraud_data.shape

(284807, 31)

In [4]:
# Look at the standard distribution of the data.
fraud_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
fraud_data.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

- The dataset doesn't possess any Nan since PCA was performed so it's ready to use. 
- Dataset is imbalanced since only .172% of transactions logged are fraudulent.
- In order to remedy the Imbalanced Class data, I will downsample the majority to improve the validity of the model.

In [3]:
# Use downsampling of the minority class to address the class imbalance. 
from sklearn.utils import resample

# Separate the majority and minority Class.
fd_majority = fraud_data[fraud_data.Class==0]
fd_minority = fraud_data[fraud_data.Class==1]


# Downsample majority class 
fd_maj_downsampled = resample(fd_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(fd_minority))    # to match minority class
                                
# Combine minority class with downsampled majority class
fd_downsampled = pd.concat([fd_maj_downsampled, fd_minority])


# Model 1: Naive Bayes

In [4]:
# Pepare to train/test split the resampled data
from sklearn.cross_validation import train_test_split

X_resampled = fd_downsampled.loc[:, fd_downsampled.columns != 'Class']
Y_resampled = fd_downsampled.loc[:, fd_downsampled.columns == 'Class']

# Used the test/train/split method with 30% holdout.
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, Y_resampled, test_size = 0.3) 

In [8]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.# Our d 
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train, y_train)

# Classify, storing the result in a new variable.
y_pred_train = bnb.predict(X_train)

# Display our results.
print(str(100*bnb.fit(X_train, y_train).score(X_test, y_test)) + '%') 

92.9054054054054%


In [9]:
# Lets check for overfitting now.
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, X_train, y_train, cv=10)


array([0.91428571, 0.88571429, 0.85714286, 0.92753623, 0.91304348,
       0.82352941, 0.89705882, 0.91176471, 0.92647059, 0.89705882])

In [13]:
# Checking the test set now.
cross_val_score(bnb, X_test, y_test, cv=10)

array([0.90322581, 0.86666667, 0.8       , 0.96666667, 1.        ,
       0.89655172, 0.86206897, 0.96551724, 0.96551724, 0.86206897])

In [21]:
# Results are consistent but lets check one last thing
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_pred_train)

array([[347,   4],
       [ 59, 278]])

# Analysis

Naive Bayes is the first model that I used because it's simple to use and is computationally inexpensive due to the level of math involved. However, NB has prerequesites before it could be used, independence and class balance. Both of these weren't an issue since PCA causes these new features to be independent and have low multicollinearity. I used downsampling in order to avoid the massive class imbalance. I thought that I had solved it since my model had an accuracy of 89.5%. I had used cross validation to make sure the variables weren't overfitting my model which worked. However, the model's accuracy was still suspicious so I used the confusion matrix to see if my model was suffering from any false positives or false negatives. My model had 4 false positives (.58%) which means that it identified a charge as fraudulent when it wasn't and failed to identify 59 charges (8.57%) that were fraudulent.  



# Model 2: Random Forest Classifier 

In [30]:
# Prepare to use the random forest and GridSearchcV
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV

rfc = ensemble.RandomForestClassifier()

#Imbalanced data
X = fraud_data.drop('Class', 1)
Y = fraud_data['Class']

# Create rfc parameters for GS-CV
param_grid = {'n_estimators': [10, 25, 50, 75, 100, 200, 500, 1000],
             'max_features': [2, 4, 6, 8],
             'max_depth': [2, 4, 6, 8]}

# Set up GS-CV
grid = GridSearchCV(rfc, param_grid, cv=5, verbose=3, n_jobs=-1)

grid.fit(X, Y)

# Show the best parameter and best score for unfiltered
print('Best Parameter for data:', grid.best_params_)
print('\nBest Score for data:', grid.best_score_)

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV] max_depth=2, max_features=2, n_estimators=10 ....................
[CV] max_depth=2, max_features=2, n_estimators=10 ....................
[CV] max_depth=2, max_features=2, n_estimators=10 ....................
[CV] max_depth=2, max_features=2, n_estimators=10 ....................
[CV] max_depth=2, max_features=2, n_estimators=10 ....................
[CV] max_depth=2, max_features=2, n_estimators=25 ....................
[CV] max_depth=2, max_features=2, n_estimators=25 ....................
[CV] max_depth=2, max_features=2, n_estimators=25 ....................
[CV]  max_depth=2, max_features=2, n_estimators=10, score=0.9987711105649381, total=   2.5s
[CV] max_depth=2, max_features=2, n_estimators=25 ....................
[CV]  max_depth=2, max_features=2, n_estimators=10, score=0.9989642217618764, total=   2.5s
[CV] max_depth=2, max_features=2, n_estimators=25 ....................
[CV]  max_depth=2, max_features=2, n_estima

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   29.9s


[CV]  max_depth=2, max_features=2, n_estimators=75, score=0.9989642217618764, total=  20.2s
[CV] max_depth=2, max_features=2, n_estimators=100 ...................
[CV]  max_depth=2, max_features=2, n_estimators=75, score=0.9985428626604168, total=  20.3s
[CV] max_depth=2, max_features=2, n_estimators=200 ...................
[CV]  max_depth=2, max_features=2, n_estimators=75, score=0.9987886448622741, total=  21.5s
[CV] max_depth=2, max_features=2, n_estimators=200 ...................
[CV]  max_depth=2, max_features=2, n_estimators=75, score=0.9982795245869981, total=  21.5s
[CV] max_depth=2, max_features=2, n_estimators=200 ...................
[CV]  max_depth=2, max_features=2, n_estimators=100, score=0.9985779993679997, total=  27.8s
[CV] max_depth=2, max_features=2, n_estimators=200 ...................
[CV]  max_depth=2, max_features=2, n_estimators=100, score=0.9988764439450862, total=  27.1s
[CV] max_depth=2, max_features=2, n_estimators=200 ...................
[CV]  max_depth=2, m

[CV] max_depth=2, max_features=4, n_estimators=500 ...................
[CV]  max_depth=2, max_features=4, n_estimators=200, score=0.9989642035778866, total= 1.4min
[CV] max_depth=2, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=2, max_features=4, n_estimators=200, score=0.9991924299081828, total= 1.4min
[CV] max_depth=2, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=2, max_features=4, n_estimators=200, score=0.9989115359632029, total= 1.4min
[CV] max_depth=2, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=2, max_features=4, n_estimators=500, score=0.9989817773252344, total= 3.4min
[CV] max_depth=2, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=2, max_features=4, n_estimators=500, score=0.9992451107756047, total= 3.3min
[CV] max_depth=2, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=2, max_features=4, n_estimators=500, score=0.9989642035778866, total= 3.3min
[CV] max_depth=2

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 21.7min


...

[CV]  max_depth=8, max_features=2, n_estimators=100, score=0.9994557775359011, total= 1.4min
[CV] max_depth=8, max_features=2, n_estimators=100 ...................
[CV]  max_depth=8, max_features=2, n_estimators=100, score=0.9990870946788153, total= 1.4min
[CV] max_depth=8, max_features=2, n_estimators=200 ...................
[CV]  max_depth=8, max_features=2, n_estimators=100, score=0.999490879724724, total= 1.4min
[CV] max_depth=8, max_features=2, n_estimators=200 ...................
[CV]  max_depth=8, max_features=2, n_estimators=100, score=0.9991748740366215, total= 1.4min
[CV] max_depth=8, max_features=2, n_estimators=200 ...................
[CV]  max_depth=8, max_features=2, n_estimators=200, score=0.9990695551420246, total= 2.7min
[CV] max_depth=8, max_features=2, n_estimators=200 ...................
[CV]  max_depth=8, max_features=2, n_estimators=200, score=0.9994557775359011, total= 2.7min
[CV] max_depth=8, max_features=2, n_estimators=200 ...................
[CV]  max_depth=8

[CV]  max_depth=8, max_features=4, n_estimators=200, score=0.9995435553526912, total= 4.9min
[CV] max_depth=8, max_features=4, n_estimators=500 ...................
[CV]  max_depth=8, max_features=4, n_estimators=200, score=0.9991573181650603, total= 4.9min
[CV] max_depth=8, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=8, max_features=4, n_estimators=200, score=0.9996664384403364, total= 4.9min
[CV] max_depth=8, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=8, max_features=4, n_estimators=200, score=0.9993855444953564, total= 4.9min
[CV] max_depth=8, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=8, max_features=4, n_estimators=500, score=0.02763245672553632, total=12.2min
[CV] max_depth=8, max_features=4, n_estimators=1000 ..................
[CV]  max_depth=8, max_features=4, n_estimators=500, score=0.9995611109160493, total=12.3min
[CV] max_depth=8, max_features=4, n_estimators=1000 ..................
[CV]  max_depth

[CV]  max_depth=8, max_features=8, n_estimators=10, score=0.9994733330992591, total=  28.5s
[CV] max_depth=8, max_features=8, n_estimators=10 ....................
[CV]  max_depth=8, max_features=8, n_estimators=10, score=0.9991924299081828, total=  28.8s
[CV] max_depth=8, max_features=8, n_estimators=25 ....................
[CV]  max_depth=8, max_features=8, n_estimators=10, score=0.9996488825687752, total=  28.6s
[CV] max_depth=8, max_features=8, n_estimators=25 ....................
[CV]  max_depth=8, max_features=8, n_estimators=10, score=0.9994382121100402, total=  28.5s
[CV] max_depth=8, max_features=8, n_estimators=25 ....................
[CV]  max_depth=8, max_features=8, n_estimators=25, score=0.005828447034865349, total= 1.2min
[CV] max_depth=8, max_features=8, n_estimators=25 ....................
[CV]  max_depth=8, max_features=8, n_estimators=25, score=0.9995962220427653, total= 1.2min
[CV] max_depth=8, max_features=8, n_estimators=25 ....................
[CV]  max_depth=8, m

[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed: 401.5min finished


Best Parameter for data: {'max_depth': 4, 'max_features': 8, 'n_estimators': 500}

Best Score for data: 0.9993293704157552


In [32]:
# Show the best parameter and best score for unfiltered since it took so long/
print('Best Parameter for data:', grid.best_params_)
print('Best Score for data:', grid.best_score_)

Best Parameter for data: {'max_depth': 4, 'max_features': 8, 'n_estimators': 500}
Best Score for data: 0.9993293704157552


In [17]:
# # Cross validate score 
# #Imbalanced data
# X = fraud_data.drop('Class', 1)
# Y = fraud_data['Class']

# # Use the best parameters from previous GS-CV
# # Took way too long so I don't want to do it again.
# rfc = ensemble.RandomForestClassifier(n_estimators=500, 
#                                          max_features=8, max_depth=4)

# # use cross val score
# cross_val_score(rfc, X, Y, cv=5)


In [15]:
# Cross validate sample set data with best parameters
X_down = fd_downsampled.drop('Class', 1)
Y_down = fd_downsampled['Class']

# Use the best parameters from previous GS-CV
# Took way too long so I don't want to do it again.
new_rfc = ensemble.RandomForestClassifier(n_estimators=500, 
                                         max_features=8, max_depth=4)

# use cross val score
cross_val_score(new_rfc, X_down, Y_down, cv=5)

array([0.92424242, 0.93434343, 0.91326531, 0.93877551, 0.91326531])

# Analyis

The random forest model for predicting fraud was extremely accurate. I could see that even my test set did pretty well using the best parameters of the raw data. The advantages of using a random forest model is that there are a lot of parameters to choose and tune which can increase the accuracy but at the same time it's the downfall of this model. It took me around 7 hours to have the results returned to me. Although the accuracy of the model is high, the fact that it's a black box model (meaning that I don't know how the model achieved it's accuracy) and it's computational time makes this model subpar.  

# Model 3: Logistic Regression

- Although there are more than 284,000 rows, I am not going to use a ridge regularization which remedies that since I'm going to use my test model to run it through the logistic regression. 
- Otherwise I would have used the ridge regularization to keep the coefficients of the logistic regression from overfitting. 
- The training model only has 984 rows so it doesn't need to penalize the coefficients.

In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

# Use GS-CV to see which alpha level is best.
parameters = {
             'C':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,10000,100000,100000000]}

lr_grid = GridSearchCV(lr, parameters, cv=5, verbose=1)

#Fit the logistic regression 
lr_grid.fit(X_down, Y_down)

#return best parameters and best score
print('Best parameters:')
print(lr_grid.best_params_)
print('Best Score:')
print(lr_grid.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Best parameters:
{'C': 0.01}
Best Score:
0.9136178861788617


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:    0.5s finished


In [19]:
# Implement parameter 
lr = LogisticRegression(C=.01)

# Cross Validate the lr model. 
cross_val_score(lr, X_down, Y_down, cv=5)

array([0.95959596, 0.93434343, 0.91326531, 0.91836735, 0.84183673])

# Logistic Regression Write Up

The logistic regression ran way faster than the random forest, but that's also because this time the GridSearch had way less parameters to run through. The accuracy of this model is 92.37% which is less accurate than the random forest model, but more accurate than the BNB model. 

The reason I used a logistic regression model instead of a linear regression model is because the data I'm trying to predict is categorical rather than continuous. 

# Model 4:  Support Vector Classifier

In [26]:
from sklearn.svm import SVC

svm = SVC()

# new parameters for this model
svc_params = [{'C': [.00001,.0001,.001,.01,.1,1,
                    10,100,1000,10000],
              'gamma': [.00001,.0001,.001,.01,.1,1,10,25,100]}]

# setting up the grid
svc_grid = GridSearchCV(svm, svc_params, cv=5, verbose=1, n_jobs=-1)

#Fit the grid
svc_grid.fit(X_down, Y_down)

#return best parameters and best score
print('Best parameters:')
print(svc_grid.best_params_)
print('Best Score:')
print(svc_grid.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 352 tasks      | elapsed:    3.6s


Best parameters:
{'C': 1e-05, 'gamma': 1e-05}
Best Score:
0.5010162601626016


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:    4.7s finished


In [28]:
# Implement the parameters parameter 
svm = SVC(C=1e-05, gamma=1e-05)

# Cross Validate the svc model. 
cross_val_score(svm, X_down, Y_down, cv=5)

array([0.49494949, 0.5       , 0.51020408, 0.5       , 0.5       ])

# Analysis

I am surprised by how bad the SVC performed since it was only able to get it half right. SVM should have tremendous accuracy due to the kernel smoothing and I think the default rbf parameter was the correct kernel to use. Once again, the job finished super quickly, but that might have something to do with there being less parameters to run through. This is the worst model so far. 

# Model 5: Gradient Boosting Model

In [29]:
# Set up Gradient boosting parameters.
gb_params = [{'n_estimators': [100,200,500,800,1000],
             'max_depth': [2,4,6]
             }]

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier()

# Use the grid
gb_grid = GridSearchCV(clf, gb_params, cv=5, verbose=1, n_jobs=-1)

# Fit the grid
gb_grid.fit(X_down, Y_down)

# Return best parameters and best score
print('Best parameters:')
print(gb_grid.best_params_)
print('Best Score:')
print(gb_grid.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   12.3s finished


Best parameters:
{'max_depth': 2, 'n_estimators': 100}
Best Score:
0.8658536585365854


In [30]:
# Implement the parameters parameter 
clf = ensemble.GradientBoostingClassifier(n_estimators=100, 
                                         max_depth=2)

# Cross Validate the gradient boosting model. 
cross_val_score(clf, X_down, Y_down, cv=5)

array([0.6969697 , 0.92929293, 0.91836735, 0.92346939, 0.8622449 ])

# Analysis

The reason I used the gradient boosing model was because it was also computationally quick. The gradient boosting model works by adjusting previous outputs based on what was learned by the old model. It learns iteratively which is why it's accuracy increases but it is also susceptible to overfitting. That's the reason I used GS_CV here too besides the fact that it can help me find the optimal parameters, it also finds it while avoiding overfitting my model. 


# Model 6: KNN

In [34]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the model
knn = KNeighborsClassifier()

# Set parameters for KNN
knn_params = [{'n_neighbors': [2,5,7,10,12,15,20]}]

# Search for the best paramters. 
knn_grid = GridSearchCV(knn, knn_params, cv=5, verbose=1, n_jobs=-1)

# Fit the grid and obtain results
knn_grid.fit(X_down, Y_down)

# Return best parameters and best score
print('Best parameters:')
print(knn_grid.best_params_)
print('Best Score:')
print(knn_grid.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best parameters:
{'n_neighbors': 2}
Best Score:
0.4532520325203252


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.2s finished


In [35]:
# Implement the parameters parameter 
knn = KNeighborsClassifier(n_neighbors=2, weights='distance')

# Cross Validate the knn model. 
cross_val_score(knn, X_down, Y_down, cv=5)

array([0.37878788, 0.38888889, 0.34183673, 0.5       , 0.35714286])

# Analysis 

The k neighbors model didn't perform as well as I thought it would. I think the main issue must be that I didn't normalize the distance of the data which is probably why the best parameter became 2 neighbors.

# Model 7: Decision Tree


In [37]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
dtc = DecisionTreeClassifier()

# Set parameters for dtc
dtc_params = [{'max_features': [2, 4, 6, 8],
             'max_depth': [2, 4, 6, 8]}]

# Search for the best paramters. 
dtc_grid = GridSearchCV(dtc, dtc_params, cv=5, verbose=1, n_jobs=-1)

# Fit the grid and obtain results
dtc_grid.fit(X_down, Y_down)

# Return best parameters and best score
print('Best parameters:')
print(dtc_grid.best_params_)
print('Best Score:')
print(dtc_grid.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters:
{'max_depth': 4, 'max_features': 8}
Best Score:
0.9115853658536586


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.2s finished


In [39]:
# Implement the parameters parameter 
dtc = DecisionTreeClassifier(max_features=8, max_depth=4)

# Cross Validate the decision tree model. 
cross_val_score(dtc, X_down, Y_down, cv=5)

array([0.55050505, 0.9040404 , 0.90816327, 0.87755102, 0.90306122])

# Analysis

The decision tree is a simpler version of the random forest since it's only one tree compared to the forest. It didn't have the opportunity to vote amongst a set of varied options so its score may be a bit lower. The model's accuracy is due to the low amount of entropy which menas that it was given enough options to correctly classify which cases were fraudulent. A drawback to this could be that I'm allowing the model to overfit but the grid search serves as a safeguard against it. 

# Conclusion

There was an incredible class imbalance initially which is why I downsized the sample so that my data could be normalized. I didn't have to intentionally look for independence and low multicollinearity due to the fact that I'm looking at principal components. 

The best model was the random forest model, but it took the longest by far. My GB model beat my logistic regression model, which beat my BNB model leaving SVC in last place. Besides the random forest model, all of the models were extremely lightweight and quick. The biggest surprise for me is that my support vector model was incredibly inaccurate, failing to predict the fraudulent cases correctly. 